In [1]:
import pandas as pd
data = pd.read_csv("Crop_Data.csv")
data.head()

,temperature,humidity,ph,water availability,season,label,Country,harvest_season
0,20.879744,82.002744,6.502985,202.935536,rainy,rice,Nigeria,spring
1,21.770462,80.319644,7.038096,226.655537,rainy,rice,Nigeria,winter
2,23.004459,82.320763,7.840207,263.964248,rainy,rice,Nigeria,rainy
3,26.491096,80.158363,6.980401,242.864034,rainy,rice,Nigeria,rainy
4,20.130175,81.604873,7.628473,262.717340,rainy,rice,Nigeria,spring


In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler

# Load the dataset
data = pd.read_csv("Crop_Data.csv")

# Encode categorical variables
label_encoder_season = LabelEncoder()
data['season'] = label_encoder_season.fit_transform(data['season'])

label_encoder_label = LabelEncoder()
data['label'] = label_encoder_label.fit_transform(data['label'])

label_encoder_country = LabelEncoder()
data['Country'] = label_encoder_country.fit_transform(data['Country'])

label_encoder_harvest_season = LabelEncoder()
data['harvest_season'] = label_encoder_harvest_season.fit_transform(data['harvest_season'])

# Split the data into features (X) and target variable (y)
X = data.drop(['label', 'harvest_season'], axis=1)
# X = data.drop(['harvest_season'], axis=1)
y_label = data['label']
y_season = data['harvest_season']

# # Standardize numerical features
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)

# Split the dataset into training and testing sets
X_train, X_test, y_label_train, y_label_test, y_season_train, y_season_test = train_test_split(
    X, y_label, y_season, test_size=0.2, random_state=42
)
# X_train, X_test, y_season_train, y_season_test = train_test_split(
#     X, y_season, test_size=0.2, random_state=42
# )

In [8]:
pip show scikit-learn 

Name: scikit-learn
Version: 1.3.2
Summary: A set of python modules for machine learning and data mining
Home-page: http://scikit-learn.org
Author: 
Author-email: 
License: new BSD
Location: c:\users\user\appdata\local\packages\pythonsoftwarefoundation.python.3.10_qbz5n2kfra8p0\localcache\local-packages\python310\site-packages
Requires: joblib, numpy, scipy, threadpoolctl
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [10]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
from joblib import dump
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# Create and train the model for crop prediction
model_label = RandomForestClassifier(n_estimators=100, random_state=42)
model_label.fit(X_train_scaled, y_label_train)

# Make predictions on the test set
y_pred_label = model_label.predict(X_test_scaled)

# Evaluate the model
accuracy_label = accuracy_score(y_label_test, y_pred_label)

# Create and train the model for crop harvest
model_harvest = RandomForestClassifier(n_estimators=100, random_state=42)
model_harvest.fit(X_train_scaled, y_season_train)

# Make predictions on the test set for harvest season
y_pred_harvest = model_harvest.predict(X_test_scaled)
accuracy_season = accuracy_score(y_season_test, y_pred_harvest)

# Precision, Recall, and F1 Score for the crop pred model
# precision_label = precision_score(y_label_test, y_pred_label, average='weighted')
# recall_label = recall_score(y_label_test, y_pred_label, average='weighted')
# f1_label = f1_score(y_label_test, y_pred_label, average='weighted')

print(f"Model Accuracy for crop pred: {accuracy_label}")
print(f"Model Accuracy for harvest pred: {accuracy_season}")
# print(f"Precision for crop pred model: {precision_label}")
# print(f"Recall for crop pred model: {recall_label}")
# print(f"F1 Score for crop pred model: {f1_label}")



# Classification Report
# print("Classification Report:")
# print(classification_report(y_label_test, y_pred_label))
crop_model_filename = 'crop_prediction_model.joblib'
harvest_model_filename = 'harvest_prediction_model.joblib'
dump({'model': model_label, 'label_encoder_season': label_encoder_season, 'label_encoder_label': label_encoder_label, 'label_encoder_country': label_encoder_country}, 'crop_prediction_model.joblib')
dump({'model': model_harvest, 'label_encoder_season': label_encoder_season, 'label_encoder_harvest_season': label_encoder_harvest_season, 'label_encoder_country': label_encoder_country}, 'harvest_prediction_model.joblib')
dump(scaler, 'standard_scaler.joblib')

print(f"Model saved as {crop_model_filename}")
print(f"Model saved as {harvest_model_filename}")


Model Accuracy for crop pred: 0.9857142857142858
Model Accuracy for harvest pred: 0.2642857142857143
Model saved as crop_prediction_model.joblib
Model saved as harvest_prediction_model.joblib


In [7]:
# Sample input values for both label and harvest season prediction
sample_input = {
    'temperature': 28.0,
    'humidity': 85.0,
    'ph': 9.0,
    'water_availability': 250.0,
    'season': 'summer',
    'country': 'South Africa'
}

# Use the loaded label encoders
season_encoded = label_encoder_season.transform([sample_input['season']])[0]
country_encoded = label_encoder_country.transform([sample_input['country']])[0]

# Standardize the sample input
sample_input_scaled = scaler.transform([[
    sample_input['temperature'],
    sample_input['humidity'],
    sample_input['ph'],
    sample_input['water_availability'],
    season_encoded,
    country_encoded,
]])

# Make predictions for crop label
crop_label_prediction = model_label.predict(sample_input_scaled)[0]
predicted_crop = label_encoder_label.inverse_transform([crop_label_prediction])[0]

# Make predictions for harvest season
harvest_season_prediction = model_harvest.predict(sample_input_scaled)[0]
predicted_harvest_season = label_encoder_harvest_season.inverse_transform([harvest_season_prediction])[0]

print(f"The predicted crop is: {predicted_crop}")
print(f"The predicted harvest season is: {predicted_harvest_season}")


The predicted crop is: rice
The predicted harvest season is: spring


C:\Users\USER\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [21]:
from keras.models import Sequential
from keras.layers import Dense

# Initialize the Neural Network model
model_season_nn = Sequential()

# Add layers to the model
model_season_nn.add(Dense(units=64, input_dim=X_train_scaled.shape[1], activation='relu'))
model_season_nn.add(Dense(units=32, activation='relu'))
model_season_nn.add(Dense(units=1, activation='linear'))  # Linear activation for regression

# Compile the model
model_season_nn.compile(optimizer='adam', loss='mean_squared_error')

# Train the Neural Network model
model_season_nn.fit(X_train_scaled, y_season_train, epochs=50, batch_size=32, validation_data=(X_test_scaled, y_season_test))


Epoch 1/50
35/35 [==============================] - 1s 10ms/step - loss: 3.1642 - val_loss: 1.6054
Epoch 2/50
35/35 [==============================] - 0s 6ms/step - loss: 1.3653 - val_loss: 1.3087
Epoch 3/50
35/35 [==============================] - 0s 6ms/step - loss: 1.2583 - val_loss: 1.2808
Epoch 4/50
35/35 [==============================] - 0s 5ms/step - loss: 1.2292 - val_loss: 1.2725
Epoch 5/50
35/35 [==============================] - 0s 4ms/step - loss: 1.2137 - val_loss: 1.2732
Epoch 6/50
35/35 [==============================] - 0s 4ms/step - loss: 1.2019 - val_loss: 1.2714
Epoch 7/50
35/35 [==============================] - 0s 4ms/step - loss: 1.1980 - val_loss: 1.2689
Epoch 8/50
35/35 [==============================] - 0s 4ms/step - loss: 1.1968 - val_loss: 1.2703
Epoch 9/50
35/35 [==============================] - 0s 4ms/step - loss: 1.1878 - val_loss: 1.2726
Epoch 10/50
35/35 [==============================] - 0s 6ms/step - loss: 1.1829 - val_loss: 1.2719
Epoch 11/50
35/35 

In [22]:
# Evaluate the Neural Network model
mse_season_nn = model_season_nn.evaluate(X_test_scaled, y_season_test)
print(f'Mean Squared Error for Season Prediction (Neural Network): {mse_season_nn}')

# Make predictions using the Neural Network model
season_predictions_nn = model_season_nn.predict(X_test_scaled)

# Calculate R-squared value for the Neural Network model
from sklearn.metrics import r2_score
r2_season_nn = r2_score(y_season_test, season_predictions_nn)
print(f'R Squared Value for the harvest model (Neural Network): {r2_season_nn}')


1/9 [==>...........................] - ETA: 0s - loss: 1.2658

9/9 [==============================] - 0s 4ms/step - loss: 1.3198
Mean Squared Error for Season Prediction (Neural Network): 1.3198050260543823
9/9 [==============================] - 0s 3ms/step
R Squared Value for the harvest model (Neural Network): -0.0563722140271965


In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
from joblib import dump
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# Create and train the model for crop prediction
model_label = RandomForestClassifier(n_estimators=100, random_state=42)
model_label.fit(X_train_label, y_train_label)

# Make predictions on the test set
y_pred_label = model_label.predict(X_test_label)

# Evaluate the model
accuracy_label = accuracy_score(y_test_label, y_pred_label)

# Create and train the model for crop harvest
model_harvest = RandomForestRegressor(n_estimators=100, random_state=42)
model_harvest.fit(X_train_harvest, y_train_harvest)

# Make predictions on the test set for harvest season
y_pred_harvest = model_harvest.predict(X_test_harvest)

# Evaluate the harvest season prediction model
mse = mean_squared_error(y_test_harvest, y_pred_harvest)
r2 = r2_score(y_test_harvest, y_pred_harvest)

# Precision, Recall, and F1 Score for the crop pred model
precision_label = precision_score(y_test_label, y_pred_label, average='weighted')
recall_label = recall_score(y_test_label, y_pred_label, average='weighted')
f1_label = f1_score(y_test_label, y_pred_label, average='weighted')

print(f"Model Accuracy for crop pred: {accuracy_label}")
print(f"Precision for crop pred model: {precision_label}")
print(f"Recall for crop pred model: {recall_label}")
print(f"F1 Score for crop pred model: {f1_label}")

print(f'Mean Squared Error for the harvest model: {mse}')
print(f'R Squared Value for the harvest model: {r2}')

# Classification Report
# print("Classification Report:")
# print(classification_report(y_test_label, y_pred_label))
# crop_model_filename = 'crop_prediction_model.joblib'
# harvest_model_filename = 'harvest_prediction_model.joblib'
# dump({'model': model_label, 'label_encoder_season': label_encoder_season, 'label_encoder_label': label_encoder_label, 'label_encoder_country': label_encoder_country}, 'crop_prediction_model.joblib')
# dump({'model': model_harvest, 'label_encoder_season': label_encoder_season, 'label_encoder_harvest_season': label_encoder_harvest_season, 'label_encoder_country': label_encoder_country}, 'harvest_prediction_model.joblib')

# print(f"Model saved as {crop_model_filename}")
# print(f"Model saved as {harvest_model_filename}")


Model Accuracy for crop pred: 0.9857142857142858
Precision for crop pred model: 0.9867770034843206
Recall for crop pred model: 0.9857142857142858
F1 Score for crop pred model: 0.9856051587301587
Mean Squared Error for the harvest model: 1.3851364285714285
R Squared Value for the harvest model: -0.10866347459443992
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        22
           1       1.00      1.00      1.00        22
           2       1.00      1.00      1.00        14
           3       0.90      1.00      0.95        18
           4       1.00      1.00      1.00        20
           5       1.00      1.00      1.00        23
           6       0.95      1.00      0.97        39
           7       1.00      1.00      1.00        22
           8       1.00      1.00      1.00        21
           9       1.00      1.00      1.00        20
          10       1.00      0.90      0.95        20
          11

In [5]:
# Sample input values for both label and harvest season prediction
sample_input = {
    'temperature': 25.0,
    'humidity': 75.0,
    'ph': 7.0,
    'water_availability': 250.0,
    'season': 'rainy',
    'country': 'Nigeria'
}

# Use the loaded label encoders
season_encoded = label_encoder_season.transform([sample_input['season']])[0]
country_encoded = label_encoder_country.transform([sample_input['country']])[0]

# Standardize the sample input
sample_input_scaled = scaler.transform([[
    sample_input['temperature'],
    sample_input['humidity'],
    sample_input['ph'],
    sample_input['water_availability'],
    season_encoded,
    country_encoded,
]])

# Make predictions for crop label
crop_label_prediction = model_label.predict(sample_input_scaled)[0]
predicted_crop = label_encoder_label.inverse_transform([crop_label_prediction])[0]

# Make predictions for harvest season
harvest_season_prediction = model_harvest.predict(sample_input_scaled)[0]
predicted_harvest_season = label_encoder_harvest_season.inverse_transform([harvest_season_prediction])[0]

print(f"The predicted crop is: {predicted_crop}")
print(f"The predicted harvest season is: {predicted_harvest_season}")


The predicted crop is: jute
The predicted harvest season is: rainy


C:\Users\USER\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [3]:
def predict_crop():
    # Get user input
    temperature = float(input("Enter temperature: "))
    humidity = float(input("Enter humidity: "))
    ph = float(input("Enter pH level: "))
    water_availability = float(input("Enter water availability: "))
    season = label_encoder_season.transform([input("Enter season: ")])[0]
    country = label_encoder_country.transform([input("Enter country: ")])[0]

    # Standardize user input
    user_input = scaler.transform([[temperature, humidity, ph, water_availability, season, country]])

    # Make a prediction
    prediction = model.predict(user_input)[0]

    # Decode the predicted crop label
    predicted_crop = label_encoder_label.inverse_transform([prediction])[0]

    print(f"The predicted crop is: {predicted_crop}")

# Call the function to make predictions
# predict_crop()


In [5]:
# Sample input values
sample_input = {
    'temperature': 18.0,
    'humidity': 65.0,
    'ph': 5.5,
    'water_availability': 200.0,
    'season': 'summer',
    'country': 'Sudan'
}

# Standardize the sample input
sample_input_scaled = scaler.transform([[
    sample_input['temperature'],
    sample_input['humidity'],
    sample_input['ph'],
    sample_input['water_availability'],
    label_encoder_season.transform([sample_input['season']])[0],  # Use the correct encoder
    label_encoder_country.transform([sample_input['country']])[0],  # Use the correct encoder
]])

# Make a prediction
sample_prediction = model_label.predict(sample_input_scaled)[0]

# Decode the predicted crop label
predicted_crop = label_encoder_label.inverse_transform([sample_prediction])[0]

print(f"The predicted crop for the sample input is: {predicted_crop}")


The predicted crop for the sample input is: pigeonpeas


C:\Users\USER\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
